<style>div.title-slide {    width: 100%;    display: flex;    flex-direction: row;            /* default value; can be omitted */    flex-wrap: nowrap;              /* default value; can be omitted */    justify-content: space-between;}</style><div class="title-slide">
<span style="float:left;">Licence CC BY-NC-ND</span>
<span>Thierry Parmentelat &amp; Arnaud Legout</span>
<span><img src="media/both-logos-small-alpha.png" style="display:inline" /></span>
</div>

# Notebooks interactifs 

In [ ]:
%pylab inline
import matplotlib.pyplot as plt
import numpy as np
plt.ion()

## Complément - niveau basique

Pour conclure cette série sur les outils de visualisation, nous allons voir quelques fonctionnalités disponibles uniquement dans l'environnement des notebooks, et qui offrent des possibilités supplémentaires par rapport aux visualisations que l'on a vues jusqu'à maintenant.

Pour commencer et en guise d'exemple, je vous invite à consulter un [exemple de notebook publié par la célèbre revue *Nature*](http://www.nature.com/news/ipython-interactive-demo-7.21492), qui pourra vous donner une idée de ce qu'il est possible de faire de cette façon; essayez de bien penser à cliquer sur `Expand to fullscreen` pour un meilleur confort.

Je vous signale d'ailleurs que [ce notebook est disponible sur github ici](https://github.com/jupyter/nature-demo).

### Une visualisation interactive simple : `interact`

Pour refaire de notre coté quelque chose d'analogue, nous allons commencer par animer la fonction sinus, avec un bouton pour régler la fréquence. Pour cela nous allons utiliser la fonction `interact`; à nouveau c'est un utilitaire qui fait partie de l'écosystème des notebooks, et plus précisément du module `ipywidgets` :

In [ ]:
from ipywidgets import interact

Ce que je fais dans un premier temps, c'est d'écrire une fonction qui prend en paramètre la fréquence, et qui dessine la fonction sinus sur un intervalle fixe de 0. à $4\pi$ :

In [ ]:
def sinus(freq):
    X = np.linspace(0., 4*np.pi, 200)
    Y = np.sin(freq*X)
    plt.plot(X, Y)

In [ ]:
sinus(1)

In [ ]:
sinus(0.5)

Maintenant, plutôt que de tracer inidividuellement les courbes une à une, j'utilise `interact` qui va m'afficher une réglette pour changer le paramètre `freq`. Ça se présente comme ceci :

In [ ]:
pylab.rcParams['figure.figsize'] = (12, 8)
interact(sinus, freq=(0.5, 10., 0.5));

## Appel à `interact`

* premier argument: une fonction `f`
* +: autant d'arguments supplémentaires
  * que de paramètres attendus par `f`
* chacun est un `Slider`

Par exemple
* si `f` prend deux aguments `foo` et `bar`
* `interact(f, foo=.., bar=..)`

## Bouche-trou: `fixed`

* Quand un des paramètres reste fixe

In [ ]:
def histo2(omega, phi):
    plt.hist(np.linspace(phi, phi+1, 100)**omega, alpha=0.3)

In [ ]:
from ipywidgets import fixed
interact(histo2, omega=(0., 2.), phi=fixed(0.))

## Sliders

* en fait cet appel à interact
  * `interact(histo, omega=(0., 2.))`
* est un raccourci pour
  * `interact(histo, omega=Floatslider(min=0., max=2.))`

Ce qui permet un peu plus de réglages:
* `min`, `max`, `step`
* et `value` - valeur initiale

In [ ]:
from ipywidgets import FloatSlider
interact(histo, omega=FloatSlider(min=0., max=2., value=1.83))

## Sliders (ctd)

Plusieurs types disponibles
* booléen (checkbox)
* texte (saisie utilisateur)
* liste ou dict (choix multiples)

In [ ]:
# une liste ou un dictionnaire est transformé(e) en un Dropdown
interact(histo, omega={'petit': 0.1, 'moyen': 0.5, 'plat': 1.0})

* [Liste complète des widgets](http://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html)
* et [notamment sur Dropdown](http://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html#Dropdown)

In [ ]:
from ipywidgets import Dropdown
interact(histo, omega=Dropdown(options={'petit': 0.1, 'moyen': 0.5, 'plat': 1.0}))

## Calculs lourds
* créer un slider avec `continuous_update=False`
* pour éviter mises à jour trop fréquentes

# Purpose

R2lab's radiomap is a set of measurements that act as a calibration of the testbed.
The goal is to measure and visualize **received power** at **all node locations** 
when a radio signal is sent **from any given sender** node.

Additionally, that same experiment is carried out with **various settings** 
for the emitted signal, like emission power, Tx rate, channel frequency, 
and with various antenna setups (single antenna, multiple antennas).

## Workflow

End to end experiment involves 2 successive stages:

1. **data acquisition** : per se, including post-processing (aggregation); this can be carried out with the `acquiremap.py` python script, or interactively through the first part of the present `visumap.ipynb` notebook
1. **visualization** : interactively, through the second part of this notebook

For convenience, this git repository also features a directory `datasample` that contains one dataset obtained by running the first-stage acquisition script, so that visualization can be performed right away, as a way to give a quick sense of the results. 

## Notebook Installation 

Here's a list of instructions for setting up the notebook tool jupyter (on OSX):

```
$ sudo pip3 install jupyter 
$ sudo pip3 install --upgrade notebook
$ jupyter nbextension enable --py --sys-prefix widgetsnbextension
$ sudo pip3 install plotly 
```

Then start a notebook server as usual
```
$ jupyter notebook
```

You may need to mention this extra option if you run an early release of jupyter-5.0
```
$ jupyter notebook --NotebookApp.iopub_data_rate_limit=10000000000
```

Likewise, on macOS there may be a need to add this option with jupyter as of June 2017, (regardless of your default browser)

```
$ jupyter notebook NotebookApp.browser=Safari
```

Once the notebook web page shows up, double click on the file `visumap.ipynb` on your browser, and evaluate the commands using "Shift-Enter" (if needed, see other Internet resources about how to use a notebook).

The notebook contains all information about its own capabilities.

# Data acquisition with `acquiremap.py`

This script is designed to expose to the outside 3 levels of scenarios:

* a `one_run` python function, that runs a complete set of measurements on all nodes, with a specific combination of environment settings (like transmission power, number of antennas, and similar)

* a `all_runs` python function, that calls `one_run` with all possible values for the environment settings

* the script itself, when invoked from the command-line, calls `all_runs` with the environment settings described as command-line options.

Additionally, all these functions can be instructed to perform node initializations (load a fresh image on all nodes, and turn off unused nodes). When this feature is turned on with the 2 multiple-runs versions (either `all_runs` or at script-level), nodes initialization is performed only once before the the first invokation of `one_run`.

In [ ]:
# for convenience, we use this notebook extension
# that will reload the python file if changed from 
# a text editor
%load_ext autoreload
%autoreload 2

## `one_run`

`one_run` performs data collection on all nodes for one given setting of the environment variables. 

In [ ]:
from acquiremap import one_run

In [ ]:
help(one_run)

##### The `one_run` function implements a scenario in which each node sends a number of ping packets to every other node.

In addition, all nodes run a tcpdump process, and at the end of the run, 
every pcap file (called `fit<N>.pcap` at node `N`) is analyzed locally at node N to 
retrieve the RSSI values received from each other node on its antenna(s), and the result 
is stored in file `result-N.txt` (still on node N).

When all nodes are done, the results are fetched from all nodes and centralized on this laptop in a directory called `t{}-r{}-a{}-ch{}` containing all the files retrieved from all nodes, i.e., `fitN.pcap`, `result-N.txt` for all N nodes.

With:

* `t{}` identifies Tx power for sender nodes in dBm (e.g. 5 to 14):
* `r{}` identifies PHY Tx rate used on all nodes (1 to 54 Mbps)
* `a{}` identifies the antenna mask on all nodes
* `ch{}` denotes the WiFi channel used for transmission

At that point, the post-processing function `processmap.py` is invoked 
to generate intermediate files `rssi-<N>.txt` (one per node),
and eventually one consolidated file `RSSI.txt`, that will be used to plot the radiomap.

#### IMPORTANT NOTE:  Both Atheros 93xx (with ath9k driver) and Intel 5300 (with iwlwifi driver) a/b/g/n NICs are now supported in these scripts. However, both cards do no have the same features and in particular, the iwlwifi driver limits the number of wireless stations in the same IBSS (parameter IWLAGN_STATION_COUNT) to a dozen! So, if you run the script with the 37 nodes, you will observe strange behaviors... Also the Intel 5300 cards are not allowed to use the 5GHz band in Ad Hoc mode. One good point is that it is possible with these cards to decrease the TX power to 0dBm (the lower bound for Atheros cards is 5dBm).

##  `all_runs`

This function simply calls `one_run` with several combinations (a cartesian product) of environment settings.

In [ ]:
from acquiremap import all_runs
help(all_runs)

For example, instead of expecting a paramater `tx_power` that is a simple strings, it expects parameter `tx_powers` that is a list of `tx_power` strings to consider. So for example

    all_runs(tx_powers=[5], phy_rates=[1, 54], antenna_masks=[3, 7])
    
would result in 4 runs of `one_run` with the 2 possible values for `phy_rates` multiplied by the 2 possible values for `antenna_masks`

## shell interface

The command-interface lets you essentially call `all_runs` directly from your shell

In [ ]:
#!./acquiremap.py --help

So by default this simple shell script will run the scenario 
with all values for Tx power, PHY Tx rate and Antenna configurations. 
Book R2lab for at least 2 hours to run it.

## Run your own

The first-level directory name for results is a parameter for you to decide on,
it defaults in the python code to `myradiomap` but you are encouraged 
to provide your own in order to isolate your results.

Tweak the following cell to run your own data collection campaign:

In [ ]:
# this plays the same role as 'datasample' -- see below -- but for your own data
datadir = "myradiomap-intel"
slicename = "inria_radiomap"
slicename = "inria_oai"

In [ ]:
# uncomment this to run the data acquisition yourself
# in the local 'myradiomap' subdir
#all_runs(run_name=datadir, tx_powers=[0,15], phy_rates=[1,54], 
#         antenna_masks=[7], channels=[1], node_ids=[1,2,3,4,5,15,20,31,36,37], wireless_driver='iwlwifi',
#         load_images = True, slicename=slicename)

# Plotting R2lab Radio-Maps

## Prerequisites

The git repository comes with a pre-populated dataset collected by us, in directory `datasample`.
This contains all the RSSI information to run this visualization.
Of course you can also collect your own RSSI data, tweak `datadir` accordingly in this case.

The `datasample` directory contains a collection of `RSSI.txt` files in the following subdirectories:

* datasample/t14-r1-a1-ch1/RSSI.txt
* datasample/t14-r1-a3-ch1/RSSI.txt
* datasample/t14-r1-a7-ch1/RSSI.txt
* datasample/t14-r54-a1-ch1/RSSI.txt
* datasample/t14-r54-a3-ch1/RSSI.txt
* datasample/t14-r54-a7-ch1/RSSI.txt
* datasample/t5-r1-a1-ch1/RSSI.txt
* datasample/t5-r1-a3-ch1/RSSI.txt
* datasample/t5-r1-a7-ch1/RSSI.txt
* datasample/t5-r54-a1-ch1/RSSI.txt
* datasample/t5-r54-a3-ch1/RSSI.txt
* datasample/t5-r54-a7-ch1/RSSI.txt

Where

* `t5` means an emission power of 5dBm

* `r1` means PHY rate=1Mbps

* `a1` means 1 single valid antenna - and so 2 values in RSSI.txt
* `a3` means 2 valid antennas - and so 3 values in RSSI.txt
* `a7` means 3 valid antennas - and so 4 value in RSSI.txt

* `ch1` means channel 1, i.e. 2412 MHz frequency

This naming scheme is implemented in a helper function in `acquiremap`:

In [ ]:
from acquiremap import naming_scheme

In [ ]:
naming_scheme('example-run', tx_power=5, phy_rate=1, antenna_mask=3, channel=40)

***

### Preparation





In [ ]:
# automatic reload of modules in case of changes
%load_ext autoreload
%autoreload 2

### Importing plotly

In [ ]:
import plotly
plotly.__version__

In [ ]:
import plotly.plotly as py
import plotly.graph_objs as go

In [ ]:
# using plotly in offline mode is a requirement 
# in interactive mode - too slow otherwise
import plotly.offline as pyoff

pyoff.init_notebook_mode()

In [ ]:
# interactive notebook widgets 
from ipywidgets import (interactive_output, fixed,
                        IntSlider, Dropdown, Layout, HBox, VBox, Text)
from IPython.display import display

### Importing a few utilities to retrieve RSSI data from files and convert it to arrays

In [ ]:
# to extract data from a RSSI file
from rssi import read_rssi

In [ ]:
# convert data into x, y and z
from r2labmap import dict_to_xyzt, dict_to_3Dxyzt

In [ ]:
# import a dictionary channel -> frequency
from channels import channel_frequency, channel_options

In [ ]:
from collections import OrderedDict

***

## Generating  2D RSSI Radio-Maps 

Let us start with some - admittedly a little abstruse - `ipywidgets` code whose purpose is only to come up with some decently compact layout for our control buttons.

In [ ]:
# this plays the same role as 'datasample' -- see below -- but for your own data
datadir = "datasample"

In [ ]:
def radiomap_dashboard():
    """
    some contorsions with ipywidgets to show controls in 
    a compact way
    create and display a dashboard 
    return a dictionary name->widget suitable for interactive_output
    """
    # dashboard pieces as widgets
    l75 = Layout(width='75%')
    l50 = Layout(width='50%')
    l32 =  Layout(width='32%')
    l25 =  Layout(width='25%')

    # all space
    w_datadir = Text(description="run name", value=datadir,
                     layout=l25)
    w_sender = IntSlider(description="sender node", 
                         min=1, max=37, step=1, value=21, 
                         continuous_update=False, layout=l75)
    w_power = Dropdown(options=list(range(0, 16)),
                       value=14, description="tx power in dBm", layout=l32)
    w_rate = Dropdown(options=[1, 54], value=1,
                      description="phy rate", layout=l32)
    # yeah, this is a little weird all right
    w_antenna_mask = Dropdown(options=OrderedDict([ ("1", 1), ("2", 3), ("3", 7)]),
                              value=7,
                              description="number of antennas: ", layout=l50)
    w_channel = Dropdown(options=channel_options,
                         value=1, description="channel", layout=l32)
    w_rssi_rank = IntSlider(min=0, max=3, value=0, 
                            description="RSSI rank: ", layout=l50)

    # update range for the rssi_rank widget from selected antenna_mask
    def update_rssi_rank_from_antennas(*args):
        w_rssi_rank.max = 3 if w_antenna_mask.value == 7\
            else 2 if w_antenna_mask.value == 3 \
            else 1
    w_antenna_mask.observe(update_rssi_rank_from_antennas, 'value')

    # make up a dashboard
    dashboard = VBox([HBox([w_datadir, w_sender]),
                      HBox([w_power, w_rate, w_channel]),
                      HBox([w_antenna_mask, w_rssi_rank])])
    display(dashboard)
    return dict(datadir=w_datadir, sender=w_sender,
                power=w_power, rate=w_rate, channel=w_channel, 
                antenna_mask=w_antenna_mask,
                rssi_rank=w_rssi_rank)

With this in place, we can define a first visualization angle that relies on a 2D representation as offered by `plotly`'s `Heatmap` tool:

In [ ]:
def radiomap2D(datadir, sender, power, rate, antenna_mask, channel, rssi_rank):
    # locate corresponding data file
    filename = str(naming_scheme(datadir, power, rate, antenna_mask, channel) / "RSSI.txt")
    # read that file
    data_dict = read_rssi(filename, sender, rssi_rank)
    if not data_dict:
        return

    X, Y, Z, T = dict_to_xyzt(data_dict)
    rssi = go.Heatmap( x=X, y=Y, z=Z, text=T, zmin=-100, zmax=0, zauto=False, opacity=1)
    data = [rssi]
    layout = go.Layout(
        title="R2lab Radio-Map: Rx power (in dBm) when fit{:02d} is transmitting<br>from {}"
              .format(sender, filename))
    figure = go.Figure(data = data, layout=layout)
    pyoff.iplot(figure)

In [ ]:
# interactively call radiomap2D
interactive_output(radiomap2D, radiomap_dashboard())

*****

***

## Generating 3D RSSI Radio-Maps

Likewise, but this time using `plotly`'s `Surface` object:

In [ ]:
def radiomap3D(datadir, sender, power, rate, antenna_mask, channel, rssi_rank):
    # locate corresponding data file
    filename = str(naming_scheme(datadir, power, rate, antenna_mask, channel) / "RSSI.txt")
    # read that file
    data_dict = read_rssi(filename, sender, rssi_rank)
    if not data_dict:
        return

    X, Y, Z, T = dict_to_3Dxyzt(data_dict)
    trace = go.Surface(x=X, y=Y, z=Z, text=T, zmin=-100, zmax=0,
                       #connectgaps=False,
                      )
    data_test = go.Data([trace])
    axis = dict(
        showbackground=True, # show axis background                                                   
        backgroundcolor="rgb(204, 204, 204)", # set background color to grey                          
        gridcolor="rgb(255, 255, 255)",       # set grid line color                                   
        zerolinecolor="rgb(255, 255, 255)",   # set zero grid line color                              
    )
    title="R2lab Radio-Map: Rx power (in dBm) when fit{:02d} is transmitting<br>from {}"\
          .format(sender, filename)
    layout = go.Layout(
        autosize=True,
        title = title,
        scene=go.Scene(                                     
            xaxis=go.XAxis(axis), # set x-axis style                                                  
            yaxis=go.YAxis(axis), # set y-axis style                                                  
            zaxis=go.ZAxis(axis, title="RSSI (dBm)  ")  # set z-axis style                                                  
        )
    )
    
    figure = go.Figure(data=data_test, layout=layout)
    pyoff.iplot(figure)

In [ ]:
# interactively call radiomap3D
interactive_output(radiomap3D, radiomap_dashboard())